In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

from setup_nyc_payroll import download_data

## Importing Dataset

In [46]:
download_data(2021, 300000, 'nyc_payroll.csv')

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
FILENAME = 'nyc_payroll.csv'
dataset = pd.read_csv(FILENAME)
# Removing columns
dataset.drop(columns=['fiscal_year', 'payroll_number', 'agency_name'])
X = df.iloc[:, :-1].values

FileNotFoundError: [Errno 2] No such file or directory: 'nyc_payroll.csv'

## Encoding Categorical Data